# Modelos Causais de Double Machine Learning (DML) e CATEs

Prof. Daniel de Abreu Pereira Uhr

## Conteúdo

* Causal Forest DML
* Orthogonal Random Forest for Causal Inference


## Referências

**Principais:**
* Microsoft EconML: https://econml.azurewebsites.net/
* UBER CausalML: https://causalml.readthedocs.io/en/latest/
* https://docs.doubleml.org/stable/index.html
* https://github.com/MasaAsami/ReproducingDMLDiD/blob/main/notebook/Reproduction_of_DMLDiD_RO_for_NEW_SIMDATA.ipynb
* Schmidheiny, K., & Siegloch, S. (2023). On event studies and distributed-lags in two-way fixed effects models: Identification, equivalence, and generalization. Journal of Applied Econometrics, 1- 19. https://doi.org/10.1002/jae.2971
* Stevenson, Betsey, Wolfers, Justin, 2006. Bargaining in the shadow of the law: Divorce laws and family distress. Q. J. Econ. 121 (1), 267–288.
* Goodman-Bacon, A. (2021). Difference-in-differences with variation in treatment timing. Journal of Econometrics. https://doi.org/10.1016/j.jeconom.2021.03.014
* Callaway, B. and Sant'Anna, P. H. C. (2021). Difference-in-Differences with multiple time periods. Journal of Econometrics. https://doi.org/10.1016/j.jeconom.2020.12.001

**Complementares:**

* Borusyak, K.; Jaravel, X. and Spiess, J. (2023). Revisiting Event Study Designs: Robust and Efficient Estimation. arXiv: https://arxiv.org/pdf/2108.12419.pdf
* Clément deChaisemartin, Xavier d’Haultfoeuille. (2022) Difference-in-Differences Estimators of Intertemporal Treatment Effects. hal-03873903
* Roth et al. (2022) What’s Trending in Difference-in-Differences? A Synthesis of the Recent Econometrics Literature. https://www.jonathandroth.com/assets/files/DiD_Review_Paper.pdf

### Introdução

Vimos o processo de ortogonalização e como obter o DML. O DML é um estimador de efeito causal que utiliza machine learning para estimar o efeito causal de um tratamento em um conjunto de dados. O DML é uma extensão do método de regressão tradicional, que permite lidar com variáveis de confusão e heterogeneidade não observada.

Em termos de aplicação prática você pode estar se perguntando: 

* ***"Quando devo usar o DML?"*** 

Então, você deve usar o DML com score ortogonal quando:

* Causalidade é seu objetivo principal,

* Alta dimensão / não-linearidade é um desafio,

* Você quer robustez estatística sem abrir mão da flexibilidade do ML.

### Causal Forest DML

Athey & Wager (2018) propose um método de aprendizado de máquina para estimar efeitos heterogêneos de tratamento (CATEs) usando florestas aleatórias. O método é chamado de Causal Forest e é uma extensão do Random Forest.

* Athey, S., & Wager, S. (2018). Estimation and Inference of Heterogeneous Treatment Effects using Random Forests. Journal of the American Statistical Association, 113(523), 230-248. https://doi.org/10.1080/01621459.2017.1319839

**Abstract:**

*"[...] desenvolvemos uma floresta causal não paramétrica para estimar efeitos heterogêneos de tratamento que estende o algoritmo de floresta aleatória amplamente utilizado de Breiman. Na estrutura de resultados potenciais com ausência de confusão, mostramos que as florestas causais são consistentes pontualmente para o efeito real do tratamento e têm uma distribuição amostral assintoticamente gaussiana e centrada. Também discutimos um método prático para construir intervalos de confiança assintóticos para o efeito real do tratamento, centrados nas estimativas da floresta causal. Nossos resultados teóricos baseiam-se em uma teoria gaussiana genérica para uma grande família de algoritmos de floresta aleatória. Até onde sabemos, este é o primeiro conjunto de resultados que permite que qualquer tipo de floresta aleatória, incluindo florestas de classificação e regressão, seja usado para inferência estatística comprovadamente válida. Em experimentos, constatamos que as florestas causais são substancialmente mais poderosas do que os métodos clássicos baseados em correspondência de vizinho mais próximo, especialmente na presença de covariáveis ​​irrelevantes. [...]"*


***Visão Geral***

Estimar o efeito médio condicional do tratamento (**CATE**):

$$
\theta(x) = \mathbb{E}[Y(1) - Y(0) \mid X = x]
$$

onde:

- $ Y(1), Y(0) $: resultados potenciais
- $ X $: covariáveis observadas
- $ D \in \{0, 1\} $: indicador de tratamento observado
- $ Y = D Y(1) + (1 - D) Y(0) $: resultado observado


***Hipóteses de Identificação***

Para identificar $ \theta(x) $, precisamos das seguintes condições:

- **Ignorabilidade condicional (unconfoundedness)**:
  
$$
(Y(1), Y(0)) \perp D \mid X
$$

- **Suporte comum (overlap)**:

$$
0 < P(D=1 \mid X=x) < 1 \quad \text{para todo } x
$$


***O Estimador de DML com Florestas Causais***

A ideia de **Double Machine Learning (DML)** é: 

- Usar Machine Learning para estimar as funções "nuisance" de alta dimensão: regressões condicionais e propensity score.
- Substituí-las num estimador ortogonal a essas funções, garantindo robustez e inferência válida.


***Etapa 1: Estimação das Funções Auxiliares***

Estime:

$$
\hat{\mu}_1(x) = \mathbb{E}[Y \mid X=x, D=1]
$$

$$
\hat{\mu}_0(x) = \mathbb{E}[Y \mid X=x, D=0]
$$

$$
\hat{e}(x) = P(D=1 \mid X=x)
$$

Com qualquer algoritmo de ML (ex: Random Forest, Lasso, etc).

***Etapa 2: Construção do Score Ortogonal (Neyman Orthogonal Score)***

$$
\psi(W_i; \theta, \eta) = \left( \frac{D_i - \hat{e}(X_i)}{\hat{e}(X_i)(1-\hat{e}(X_i))} \right) \cdot (Y_i - \hat{\mu}_{D_i}(X_i)) + \hat{\mu}_1(X_i) - \hat{\mu}_0(X_i) - \theta(X_i)
$$

A condição de ortogonalidade:

$$
\frac{\partial \mathbb{E}[\psi(W; \theta, \eta)]}{\partial \eta} \Big|_{\eta=\eta_0} = 0
$$

garante que pequenos erros em $ \eta $ (nuisance functions) não impactam assintoticamente a inferência sobre $ \theta $.

**Explicação Detalhada:**

O que cada termo representa?

| Símbolo | Significado |
|:--------|:------------|
| $ W_i $ | Todas as variáveis observadas: $ (Y_i, D_i, X_i) $ |
| $ \theta $ | Função alvo: o efeito condicional que queremos estimar, $ \theta(x) = \mathbb{E}[Y(1) - Y(0) \mid X=x] $ |
| $ \eta $ | Conjunto de funções "nuisance": $ (\mu_0(x), \mu_1(x), e(x)) $ |
| $ D_i $ | Variável de tratamento observada (0 ou 1) |
| $ Y_i $ | Resultado observado |
| $ \hat{e}(X_i) $ | Propensity score estimado: $ \mathbb{P}(D=1 \mid X=x) $ |
| $ \hat{\mu}_1(X_i) $, $ \hat{\mu}_0(X_i) $ | Resultados esperados estimados sob tratamento e controle |

***1. Primeiro termo: resíduo ajustado pelo propensity score***

$$
\left( \frac{D_i - \hat{e}(X_i)}{\hat{e}(X_i)(1-\hat{e}(X_i))} \right) \cdot (Y_i - \hat{\mu}_{D_i}(X_i))
$$

- $ D_i - \hat{e}(X_i) $ → diferença entre o tratamento observado e a propensão prevista.
- $ Y_i - \hat{\mu}_{D_i}(X_i) $ → resíduo do resultado observado em relação ao previsto.
- Dividimos pela variância binomial $ \hat{e}(X_i)(1-\hat{e}(X_i)) $ para normalizar.

Ou seja, este termo ajusta o erro entre observado e previsto, ponderado pela incerteza de receber o tratamento.


***2. Segundo termo: diferença de médias preditas entre tratado e controle***

$$
\hat{\mu}_1(X_i) - \hat{\mu}_0(X_i)
$$

- Representa a diferença nos valores esperados do resultado sob tratamento e sob controle, para o mesmo $ X_i $. Ele captura o efeito esperado com base apenas nos modelos auxiliares.


***3. Terceiro termo: subtração do parâmetro-alvo***

$$
-\theta(X_i)
$$

- Remove $ \theta(X_i) $ para formar uma equação cujo valor esperado é zero quando $ \theta $ está corretamente estimado.

**Resumo**: Força o momento a ser "zero" na verdade populacional.

***Intuição Geral***

**Essa função $ \psi $** é um **score ortogonal**:

- É construída para ser **insensível a pequenos erros na estimação de $ \mu_0, \mu_1, e $**.
- Quando resolvemos $ \mathbb{E}[\psi(W_i; \theta, \eta)] = 0 $, obtemos um estimador consistente para $ \theta(X) $.
- O **primeiro pedaço** compensa o viés de seleção no tratamento (propensity correction).
- O **segundo pedaço** usa predições para corrigir viés diretamente.
- O **terceiro pedaço** ajusta para encontrar $ \theta(X) $ de forma que o momento seja zero.
- A função $ \psi $ torna possível usar Machine Learning para estimar funções auxiliares sem sacrificar inferência causal válida.
- Pequenos erros nas funções de regressão ou propensity não prejudicam a consistência de $ \theta(x) $.


**Propriedades da Causal Forest:**

- **Consistência** sob condições regulares (ex: suavidade de $ \theta(x) $, honestidade das árvores).
- **Normalidade assintótica ponto a ponto**:

$$
\sqrt{n}(\hat{\theta}(x) - \theta(x)) \overset{d}{\to} N(0, \sigma^2(x))
$$


***Teoria Asintótica***

Suponha:

- Estimação de $ \hat{\mu}_0, \hat{\mu}_1, \hat{e} $ convergente a taxas $ o_p(n^{-1/4}) $,
- Regularidade do modelo causal,
- Cross-fitting.

Então, o estimador $ \hat{\theta}(x) $ obtido por DML satisfaz:

$$
\sqrt{n}(\hat{\theta}(x) - \theta(x)) \overset{d}{\to} N(0, V(x))
$$

Este resultado permite construir **intervalos de confiança válidos**, algo que métodos de Machine Learning puros geralmente não permitem.


***Quando usar Causal Forest?***

Ela é especialmente útil em DML por dois motivos:

- Desenha partições do espaço de $ X $ de forma adaptada ao problema de inferência causal (não apenas para predição).
- Fornece um estimador localmente adaptativo de $ \theta(x) $ com inferência válida.


O DML com Causal Forest:

- Combina **flexibilidade não paramétrica** com **rigor estatístico**.
- Estima **efeitos heterogêneos**, essencial para avaliações de políticas públicas, economia do trabalho, entre outros.
- Permite usar ferramentas modernas (ML) **sem abrir mão da fundamentação clássica da inferência causal**.

***Quando não usar Causal Forest?***

- Se o tratamento é homogêneo e a heterogeneidade não interessa.
- Se o número de observações é muito pequeno (árvores precisam de dados para dividir bem).
- Se a interpretação de $ \theta(x) $ para cada indivíduo não for relevante no seu contexto.

***Orthogonal Random Forest for Causal Inference***

Oprescu, Syrgkanis e Wu (2019)

Quando usar o ORF em vez do Causal Forest?

Situação	Melhor escolha
Precisa de máxima robustez a erros de estimação auxiliar	✅ ORF
Quer CATEs em alta dimensão com inferência	✅ ORF
Precisa de interpretação mais simples, rápida	Causal Forest
O ORF é mais rigoroso teoricamente do que o Causal Forest "puro", mas é também mais pesado computacionalmente.

Orthogonal Random Forest for Causal Inference"
📜 Oprescu, Syrgkanis, Wu (2019)

Este é o artigo central que apresenta o modelo de Orthogonal Random Forest.

Introduz a ideia de combinar:

Score Neyman-ortogonal (como no DML)

Random Forests honestas e adaptativas para aprender o CATE 
𝜃
(
𝑥
)
θ(x).

Mostra teoria de:

Consistência ponto a ponto

Normalidade assintótica

Taxas de convergência em alta dimensão

### Double Robust Machine Learning (DRML)

O estimador duplamente robusto para o Conditional Average Treatment Effect (CATE) combina modelos de *Outcome Regression* e de *escore de propensão* para fornecer estimativas consistentes, mesmo que um dos modelos seja mal especificado. A fórmula geral é:

$$
\hat{\tau}(X_i) = \hat{\mu}_1(X_i) - \hat{\mu}_0(X_i) + \frac{D_i - \hat{p}(X_i)}{\hat{p}(X_i)(1 - \hat{p}(X_i))}(Y_i - \hat{\mu}_{D_i}(X_i)),
$$

onde:
- $ \hat{\mu}_d(X_i) = \mathbb{E}[Y_i \mid X_i, D_i = d] $ é a estimativa do desfecho esperado para o tratamento $ d $ dado $ X_i $
- $ \hat{p}(X_i) = \mathbb{P}(D_i = 1 \mid X_i) $ é a estimativa do escore de propensão, ou seja, a probabilidade de receber o tratamento dado $ X_i $
- $ D_i $ é a variável indicadora de tratamento (1 se tratado, 0 caso contrário),
- $ Y_i $ é o desfecho observado.

---

### Processo de Estimação

#### Primeiro Estágio: Estimação das Funções Incômodas

1. **Modelo de Desfecho ($ \hat{\mu}_d(X_i) $)**:  
   Estima-se a relação entre as covariáveis $ X_i $ e o desfecho $ Y_i $ para cada nível de tratamento $ d $.  
   Técnicas de aprendizado de máquina, como regressão linear, árvores de decisão ou redes neurais, podem ser utilizadas.

2. **Modelo de Escore de Propensão ($ \hat{p}(X_i) $)**:  
   Estima-se a probabilidade de um indivíduo receber o tratamento dado $ X_i $.  
   Modelos como regressão logística ou métodos de classificação podem ser aplicados.

#### Segundo Estágio: Cálculo do Estimador Duplamente Robusto

Utiliza-se a fórmula acima para calcular $ \hat{\tau}(X_i) $ para cada indivíduo, combinando as estimativas dos modelos de desfecho e de escore de propensão.

---

### Propriedades do Estimador Duplamente Robusto

1. **Robustez Dupla**:  
   O estimador $ \hat{\tau}(X_i) $ é consistente se pelo menos um dos modelos (desfecho ou escore de propensão) for corretamente especificado.  
   Isso significa que erros em um modelo podem ser compensados pela precisão do outro.

2. **Eficiência**:  
   Quando ambos os modelos são corretamente especificados, o estimador atinge eficiência assintótica, resultando em estimativas mais precisas.

---

### Implementação Prática

Na prática, é comum utilizar técnicas de *cross-fitting* para evitar viés de superajuste. O procedimento envolve dividir os dados em folds, ajustando os modelos de desfecho e escore de propensão em uma parte dos dados e validando em outra, garantindo que as estimativas sejam independentes dos dados usados para ajuste.


### Double Robust Machine Learning - DRML

O Estimador de DRML para o Conditional Average Treatment Effect (CATE) utiliza uma abordagem duplamente robusta para controlar o viés de seleção do tratamento. O estimador segue um processo de dois estágios: no primeiro, estimam-se as funções incômodas usando técnicas de *crossfitting*, e, no segundo, estima-se o CATE a partir das funções ajustadas.

O estimador pode ser descrito pelas seguintes equações:

#### Estimativa Duplamente Robusta para $ Y_{t}^{DR} $

$$ \hat{Y}_{i,t}^{DR} = \mu_t(X_i, W_i) + \frac{D_i}{\hat{p}_t(X_i, W_i)} \cdot (Y_i - \mu_t(X_i, W_i)), $$

onde:
- $ \mu_t(X_i, W_i) = \mathbb{E}[Y | X_i, W_i, T_i = t] $ é a função de resultado condicional,
- $ \hat{p}_t(X_i, W_i) = \mathbb{P}[T_i = t | X_i, W_i] $ é o escore de propensão condicional,
- $ D_i = 1[T_i = t] $ é uma variável indicadora para o tratamento $ t $

#### Estimativa do CATE

$$ \theta_t(X_i) = \mathbb{E}[\hat{Y}_{i,t}^{DR} - \hat{Y}_{i,0}^{DR} | X_i], $$

onde $ \hat{Y}_{i,0}^{DR} $ é o contrafactual correspondente ao tratamento de referência $ t = 0 $

---

### Processo de Estimação

1. **Primeiro estágio**:  
   Estimam-se as funções incômodas $ \mu_t(X, W) $ e $ \hat{p}_t(X, W) $ usando técnicas de aprendizado de máquina em um esquema de *crossfitting* para evitar viés.

2. **Segundo estágio**:  
   Substitui-se as estimativas obtidas no primeiro estágio nas equações acima e calcula-se o CATE ajustado para cada tratamento $ t $ resolvendo a regressão de $ \hat{Y}_{i,t}^{DR} - \hat{Y}_{i,0}^{DR} $ sobre $ X_i $

---

### Observações

- **Robustez**: A abordagem é robusta contra erros na especificação de $ \mu_t(X, W) $ ou $ \hat{p}_t(X, W) $, desde que pelo menos uma delas seja corretamente especificada.
- **Viés residual**: A precisão da estimativa depende fortemente da qualidade das funções incômodas ajustadas no primeiro estágio.


### Double Robust Machine Learning - DRML

O Estimador de DRML para o CATE usa técnicas de correção duplamente robustas para controlar o viés de seleção do tratamento. O estimador segue um processo de dois estágios, onde um conjunto de funções incômodas (não especificado, mas que deve ser considerado no modelo) são estimadas no primeiro estágio de uma maneira de *crossfitting* e no segundo estágio estima-se o modelo CATE. 

Ou seja, neste estimador, o CATE é estimado usando as seguintes equações:

$$ Y_{i,t}^{DR} = E[Y|X_{i}, W_{i}, T_{i}=t] + \frac{Y_{i} - E[Y|X_{i}, W_{i}, T_{i}=t]}{Pr[T_{i}=t|X_{i}, W_{i}]}.1[T_{i}=t] $$
 
Então,

$$ E[Y_{i,t}^{DR}-Y_{i,0}^{DR}|X_{i}] = \theta_{t}(X_{i}) $$


Então, se estimarmos as funções $E[Y|X, W, T]$ e $Pr[T|X, W]$ no primeiro estágio, podemos estimar o segundo estágio CATE para cada tratamento $t$, rodando uma regressão $Y_{i,t}^{DR} - Y_{i,0}^{DR}$ sobre $X$.

Os possíveis problemas de estimar a função do propensity score recai na definição arbitrária do modelo de "classificação". A segunda função é uma regressão simples em $Y$.


https://econml.azurewebsites.net/_autosummary/econml.dr.DRLearner.html


Outros Modelos Duplamente robustos similares podem ser aplicados:
* econml.dr.DRLearner (visto aqui)
  * Estimador CATE que usa técnicas de correção duplamente robustas para contabilizar o viés de seleção em variáveis observáveis no tratamento.
* econml.dr.LinearDRLearner
  * Caso especial em DRLearner que o estágio final é uma Regressão Linear em um conjunto de recursos de baixa dimensão.
* econml.dr.SparseLinearDRLearner
  * Caso especial em DRLearner que o estágio final é uma Regressão Lasso Desviada.
* econml.dr.ForestDRLearner
  * Instância do DRLearner com a RegressionForest como modelo final, para permitir inferência não paramétrica.

### Doble Robust Machine Learning Difference-in-Differences (DRMLDiD)

**É possivel utilizar DML com dados em painel?**

Antes de avançar para o DRMLDiD, vamos primeiro entender como o DML pode ser aplicado a dados em painel (DMLDiD).







Vamos considerar o efeito do homogêneo do tratamento em um painel de dados. No estilo de Sant´Anna e Zhao (2020).

Os modelos de diferença em diferenças (DID) implementados no pacote focam no caso de tratamento binário com dois períodos de tratamento. Adotando a notação de Sant'Anna e Zhao (2020) , deixe ser $Y_{it}$ o resultado de interesse para a unidade $i$ no tempo $t$. Além disso, deixe $D_{it}=1$ indicar se unidade $i$ é tratada antes do tempo $t$ (de outra forma $D_{it}=0$). Como todas as unidades começam como não tratadas ($D_{it}=0$), definir $D_{i0}=0$. Com base na notação de resultado potencial, denote $Y_{it}(1)$ como resultado da unidade $i$ no tempo $t$ se a unidade não recebeu tratamento até o momento $t$ e analogamente para $Y_{it}(0)$ com tratamento. Consequentemente, o resultado observado para a unidade $i$ no tempo $t$ é $Y_{it}=Y_{it}(1)D_{it}+Y_{it}(0)(1-D_{it})$. Além disso, deixe $X_{it}$ ser um vetor de covariáveis ​​de pré-tratamento.

O parâmetro de interesse é o efeito médio do tratamento no indivíduo tratado (ATTE). 

$$ \theta_{ATTE} = E[Y_{i1}(1) - Y_{i1}(0)|D_{it}=1]$$

As suposições de identificação correspondentes são:
* (Cond.) Tendências paralelas: $Y_{it}(1), Y_{it}(0) \perp D_{it}|X_{it}$ para $t=1,2$.
* Sobreposição: $0 < P(D_{it}=1|X_{it}) < 1$ para $t=1,2$.


Se os dados do painel estiverem disponíveis, as observações são consideradas iid. de forma ($Y_{i0},Y_{i1},D_{i},X_{i}$). Obseve que a diferença $\Delta Y_{i} = Y_{i1} - Y_{i0}$ tem que ser definida como o resultado yno DoubleMLDataobjeto.

O DoubleMLIDID implementa modelos de diferença em diferenças para dados de painel. A estimativa é conduzida por meio de seu fit() método:

In [1]:
import numpy as np
import doubleml as dml
from doubleml.datasets import make_did_SZ2020
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

ml_g = RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_leaf=5)
ml_m = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_leaf=5)

np.random.seed(42)
data = make_did_SZ2020(n_obs=500, return_type='DataFrame')
obj_dml_data = dml.DoubleMLData(data, 'y', 'd')
dml_did_obj = dml.DoubleMLDID(obj_dml_data, ml_g, ml_m)

print(dml_did_obj.fit())

================== DoubleMLDID Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['Z1', 'Z2', 'Z3', 'Z4']
Instrument variable(s): None
No. Observations: 500

------------------ Score & algorithm ------------------
Score function: observational

------------------ Machine learner   ------------------
Learner ml_g: RandomForestRegressor(max_depth=5, min_samples_leaf=5)
Learner ml_m: RandomForestClassifier(max_depth=5, min_samples_leaf=5)
Out-of-sample Performance:
Regression:
Learner ml_g0 RMSE: [[16.27429763]]
Learner ml_g1 RMSE: [[13.35731523]]
Classification:
Learner ml_m Log Loss: [[0.66601815]]

------------------ Resampling        ------------------
No. folds: 5
No. repeated sample splits: 1

------------------ Fit summary       ------------------
       coef   std err         t     P>|t|     2.5 %    97.5 %
d -2.840718  1.760386 -1.613691  0.106595 -6.291011  0.609575


### Double Robust Machine Learning - DRML

O Estimador de DRML para o CATE usa técnicas de correção duplamente robustas para controlar o viés de seleção do tratamento. O estimador segue um processo de dois estágios, onde um conjunto de funções incômodas (não especificado, mas que deve ser considerado no modelo) são estimadas no primeiro estágio de uma maneira de *crossfitting* e no segundo estágio estima-se o modelo CATE. 

Ou seja, neste estimador, o CATE é estimado usando as seguintes equações:

$$ Y_{i,t}^{DR} = E[Y|X_{i}, W_{i}, T_{i}=t] + \frac{Y_{i} - E[Y|X_{i}, W_{i}, T_{i}=t]}{Pr[T_{i}=t|X_{i}, W_{i}]}.1[T_{i}=t] $$
 
Então,

$$ E[Y_{i,t}^{DR}-Y_{i,0}^{DR}|X_{i}] = \theta_{t}(X_{i}) $$


Então, se estimarmos as funções $E[Y|X, W, T]$ e $Pr[T|X, W]$ no primeiro estágio, podemos estimar o segundo estágio CATE para cada tratamento $t$, rodando uma regressão $Y_{i,t}^{DR} - Y_{i,0}^{DR}$ sobre $X$.

Os possíveis problemas de estimar a função do propensity score recai na definição arbitrária do modelo de "classificação". A segunda função é uma regressão simples em $Y$.


https://econml.azurewebsites.net/_autosummary/econml.dr.DRLearner.html


Outros Modelos Duplamente robustos similares podem ser aplicados:
* econml.dr.DRLearner (visto aqui)
  * Estimador CATE que usa técnicas de correção duplamente robustas para contabilizar o viés de seleção em variáveis observáveis no tratamento.
* econml.dr.LinearDRLearner
  * Caso especial em DRLearner que o estágio final é uma Regressão Linear em um conjunto de recursos de baixa dimensão.
* econml.dr.SparseLinearDRLearner
  * Caso especial em DRLearner que o estágio final é uma Regressão Lasso Desviada.
* econml.dr.ForestDRLearner
  * Instância do DRLearner com a RegressionForest como modelo final, para permitir inferência não paramétrica.

#### Resumo Comparativo: DML vs DRML

* DML: Foca na remoção de viés através de uma abordagem em dois estágios, utilizando previsões de aprendizado de máquina e resíduos. Busca a ortogonalidade entre o tratamento e o desfecho.
* DRML: Utiliza a ponderação de probabilidade inversa e a modelagem do desfecho para obter estimativas robustas. É duplamente robusta porque requer que apenas um dos dois modelos esteja corretamente especificado.

Ambas as metodologias são úteis em inferência causal e se beneficiam da flexibilidade e poder dos métodos de aprendizado de máquina, mas cada uma tem suas especificidades e contextos de aplicação onde são mais adequadas.